In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "MOVIE-LENS")
sc.setLogLevel("WARN")

In [3]:
rdd_ratings = sc.textFile("hdfs://192.168.93.128:9000/input/movie_lens/ratings.csv")
rdd_movies = sc.textFile("hdfs://192.168.93.128:9000/input/movie_lens/movies.csv")

In [4]:
ratings_header = rdd_ratings.first()
ratings_rdd = (
    rdd_ratings
    .filter(lambda x: x != ratings_header)
    .map(lambda x: tuple(x.split(",")))
)

movies_header = rdd_movies.first()
movies_rdd = (
    rdd_movies
    .filter(lambda x: x != movies_header)
    .map(lambda x: tuple(x.split(",")))
)

In [5]:
ratings_header, movies_header

('userId,movieId,rating,timestamp', 'movieId,title,genres')

In [6]:
ratings_rdd.first(), movies_rdd.first()

(('1', '1', '4.0', '964982703'),
 ('1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'))

In [7]:
ratings_rdd.map(lambda x: (x[1], (int(x[0]), float(x[2]), int(x[-1])))).take(4)


[('1', (1, 4.0, 964982703)),
 ('3', (1, 4.0, 964981247)),
 ('6', (1, 4.0, 964982224)),
 ('47', (1, 5.0, 964983815))]

In [8]:
ratings_rdd = ratings_rdd.map(lambda x: (x[1], (int(x[0]), float(x[2]), int(x[-1]))))

movies_rdd = movies_rdd.map(lambda x: (x[0], (x[1], x[2])))

In [9]:
ratings_rdd.first()

('1', (1, 4.0, 964982703))

In [10]:
movies_rdd.first()

('1', ('Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'))

In [11]:
ratings_rdd.count(), movies_rdd.count()

(100836, 9742)

In [12]:
movies_rdd.filter(lambda x: x[0] == '1').collect()

[('1', ('Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'))]

In [13]:
len(ratings_rdd.countByKey())

9724